In [48]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras import Model
from tensorflow.keras.layers import Input, Dense, SimpleRNN
from tensorflow.keras.layers import Layer
from tensorflow.keras.metrics import mean_squared_error
from tensorflow.keras.models import Sequential
import numpy as np
import tensorflow.keras.backend as K

In [49]:
def get_data(path):
    data = []
    for i in ["train", "val", "test"]:
        data.append(pd.read_csv(path + i + ".csv", index_col="Datetime"))
    return data[0], data[1], data[2]
# Load data
#path = '../data/clean_data/univariate_Q_Kallveit/'
train = pd.read_csv("../data/clean_data/univariate_Q_Kallveit/univariate_Q_Kallveit.csv", index_col="Datetime")

In [50]:
def format_data(time_steps, train_percent, scale_data=True):
    dat = train.values
    Y_ind = np.arange(time_steps, len(dat), 1)
    Y = dat[Y_ind]
    rows_x = len(Y)
    X = dat[0:rows_x]
    for i in range(time_steps-1):
        temp = dat[i+1:rows_x+i+1]
        X = np.column_stack((X, temp))
    # random permutation with fixed seed
    rand = np.random.RandomState(seed=13)
    idx = rand.permutation(rows_x)
    split = int(train_percent*rows_x)
    train_ind = idx[0:split]
    test_ind = idx[split:]
    trainX = X[train_ind]
    trainY = Y[train_ind]
    testX = X[test_ind]
    testY = Y[test_ind]
    trainX = np.reshape(trainX, (len(trainX), time_steps, 1))
    testX = np.reshape(testX, (len(testX), time_steps, 1))
    return trainX, trainY, testX, testY

In [51]:
# Set up parameters
time_steps = 20
hidden_units = 4
epochs = 30
batch_size = 128 # 32, 64, 128, 256


# Create a traditional RNN network
def create_RNN(hidden_units, dense_units, input_shape, activation):
    model = Sequential()
    model.add(SimpleRNN(hidden_units, input_shape=input_shape, activation=activation[0]))
    model.add(Dense(units=dense_units, activation=activation[1]))
    model.compile(loss='mse', optimizer='adam')
    return model

model_RNN = create_RNN(hidden_units=hidden_units, dense_units=1,
                       input_shape=(time_steps,1), activation=['tanh', 'tanh'])

In [52]:
# Generate the dataset for the network
trainX, trainY, testX, testY  = format_data(time_steps, 0.7)
# Train the network
model_RNN.fit(trainX, trainY, epochs=epochs, batch_size=batch_size, verbose=2)

# Evalute model
train_mse = model_RNN.evaluate(trainX, trainY)
test_mse = model_RNN.evaluate(testX, testY)

Epoch 1/30
346/346 - 1s - loss: 87.6886 - 1s/epoch - 3ms/step
Epoch 2/30
346/346 - 1s - loss: 87.5123 - 541ms/epoch - 2ms/step
Epoch 3/30
346/346 - 1s - loss: 87.4886 - 551ms/epoch - 2ms/step
Epoch 4/30
346/346 - 1s - loss: 87.4864 - 520ms/epoch - 2ms/step
Epoch 5/30
346/346 - 1s - loss: 87.4849 - 561ms/epoch - 2ms/step
Epoch 6/30
346/346 - 1s - loss: 87.4837 - 516ms/epoch - 1ms/step
Epoch 7/30
346/346 - 1s - loss: 87.4828 - 521ms/epoch - 2ms/step
Epoch 8/30
346/346 - 1s - loss: 87.4822 - 519ms/epoch - 2ms/step
Epoch 9/30
346/346 - 1s - loss: 87.4816 - 539ms/epoch - 2ms/step
Epoch 10/30
346/346 - 1s - loss: 87.4812 - 522ms/epoch - 2ms/step
Epoch 11/30
346/346 - 1s - loss: 87.4809 - 519ms/epoch - 2ms/step
Epoch 12/30
346/346 - 1s - loss: 87.4806 - 538ms/epoch - 2ms/step
Epoch 13/30
346/346 - 1s - loss: 87.4804 - 550ms/epoch - 2ms/step
Epoch 14/30
346/346 - 1s - loss: 87.4803 - 578ms/epoch - 2ms/step
Epoch 15/30
346/346 - 1s - loss: 87.4801 - 521ms/epoch - 2ms/step
Epoch 16/30
346/346 - 

In [53]:
# Print error
print("Train set MSE = ", train_mse)
print("Test set MSE = ", test_mse)

Train set MSE =  87.47956085205078
Test set MSE =  87.84986114501953


In [54]:
# Add attention layer to the deep learning network
class attention(Layer):
    def __init__(self,**kwargs):
        super().__init__(**kwargs)

    def build(self,input_shape):
        self.W=self.add_weight(name='attention_weight', shape=(input_shape[-1],1),
                               initializer='random_normal', trainable=True)
        self.b=self.add_weight(name='attention_bias', shape=(input_shape[1],1),
                               initializer='zeros', trainable=True)
        super().build(input_shape)

    def call(self,x):
        # Alignment scores. Pass them through tanh function
        e = K.tanh(K.dot(x,self.W)+self.b)
        # Remove dimension of size 1
        e = K.squeeze(e, axis=-1)
        # Compute the weights
        alpha = K.softmax(e)
        # Reshape to tensorFlow format
        alpha = K.expand_dims(alpha, axis=-1)
        # Compute the context vector
        context = x * alpha
        context = K.sum(context, axis=1)
        return context

In [55]:
def create_RNN_with_attention(hidden_units, dense_units, input_shape, activation):
    x = Input(shape=input_shape)
    RNN_layer = SimpleRNN(hidden_units, return_sequences=True, activation=activation)(x)
    attention_layer = attention()(RNN_layer)
    outputs = Dense(dense_units, trainable=True, activation=activation)(attention_layer)
    model = Model(x,outputs)
    model.compile(loss='mse', optimizer='adam')
    return model

In [56]:
# Create the model with attention, train and evaluate
model_attention = create_RNN_with_attention(hidden_units=hidden_units, dense_units=1,
                                  input_shape=(time_steps,1), activation='tanh')
model_attention.summary()
model_attention.fit(trainX, trainY, epochs=epochs, batch_size=batch_size, verbose=2)

Model: "model_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_7 (InputLayer)        [(None, 20, 1)]           0         
                                                                 
 simple_rnn_11 (SimpleRNN)   (None, 20, 4)             24        
                                                                 
 attention_6 (attention)     (None, 4)                 24        
                                                                 
 dense_11 (Dense)            (None, 1)                 5         
                                                                 
Total params: 53
Trainable params: 53
Non-trainable params: 0
_________________________________________________________________
Epoch 1/30
346/346 - 2s - loss: 90.1783 - 2s/epoch - 5ms/step
Epoch 2/30
346/346 - 1s - loss: 87.6993 - 558ms/epoch - 2ms/step
Epoch 3/30
346/346 - 1s - loss: 87.6239 - 580ms/epoch - 2ms/step
Epo

In [57]:
# Evalute model
train_mse_attn = model_attention.evaluate(trainX, trainY)
test_mse_attn = model_attention.evaluate(testX, testY)

# Print error
print("Train set MSE with attention = ", train_mse_attn)
print("Test set MSE with attention = ", test_mse_attn)

594/594 [==============================] - 1s 939us/step - loss: 87.8502
Train set MSE with attention =  87.47994232177734
Test set MSE with attention =  87.85021209716797
